# Aprendizagem de Máquina I

## Hugo Tremonte de Carvalho

#### hugo@dme.ufrj.br

O objetivo dessa atividade é resolver um problema de regressão linear em uma base de dados real. Mais especificamente, o objetivo é trabalhar sobre [esta base do Kaggle](https://www.kaggle.com/datasets/yasserh/song-popularity-dataset/), que relaciona a popularidade de uma música com certos atributos extraídos da mesma. Dessa forma, queremos ser capazes de prever se uma determinada música será ou não popular com base em determinadas informações acústicas e informações musicais.

Esse conjunto de dados consiste de $n = 18.835$ observações e um total de $p = 13$ atributos. Note que ainda não estamos em um cenário tão desfavorável de modo que tenhamos $p \approx n$. A popularidade da música, representada na coluna de nome `song_popularity`, é um número inteiro medido entre $0$ e $100$, e é tal quantidade que deve ser prevista por você com base nos $p = 13$ atributos. Abaixo seguem os atributos que são quantitativos:
* `song_duration_ms`
* `acousticness`
* `danceability`
* `energy`
* `instrumentalness`
* `liveness`
* `loudness`
* `speechiness`
* `audio_valence`

e esses são os atributos qualitativos (**não** há uma relação de ordem entre os valores que elas assumem; usá-las ingenuamente em um problema de regressão linear é uma falha grave):
* Tonalidade da música (`key`)
* Se a música está em modo maior ou modo menor (`audio_mode`)
* Fórmula de compasso da música (`time_signature`).

Sobre a interpretação e aquisição dos atributos, todos os que estamos mantendo são bastante fáceis de serem calculados a partir de um arquivo `.mp3` contendo uma música. A interpretação deles é algo bem próximo do que o próprio nome sugere (a bruxaria está em *como* essas contas são feitas!). Veja [aqui](https://www.kaggle.com/datasets/yasserh/song-popularity-dataset/discussion/331001) mais detalhes sobre os atributos.

Portanto, a nossa missão para esta atividade é utilizar as técnicas de regressão que estudamos até o momento para encontrar uma boa forma de prever a variável resposta a partir dos atributos. Para isso, siga o roteiro abaixo:
* Faça uma análise exploratória para entneder como os seus atributos se comportam, se há multicolinearidade, se algum deles é bastante correlacionado com a resposta, etc.
* Note que alguns atributos têm uma ordem de grandeza bem discrepante, o que pode ser numericamente problemático. Iremos tratar disso na terceira parte dessa aula.
* Separe o seu conjunto de dados em treinamento e teste
* Faça ajustes dos modelos de regressão que aprendemos, implementando alguma busca por validação cruzada no conjunto de treinamento para encontrar hiperparâmetros ótimos, quando for pertinente.
* Avalie o desempenho do seu modelo no conjunto de teste, e reporte os seus resultados.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as r2

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

## Leitura dos dados e análise exploratória

In [ ]:
df = pd.read_csv("song_data.csv")
print(df.shape)
df.head()

In [ ]:
# Verificando se há dados faltantes
# df.isnull()
df.isnull().sum()

In [ ]:
# Verificando se há linhas duplicadas
# df.duplicated()
df.duplicated().sum()

In [ ]:
# Removendo linhas duplicadas
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
# ATENÇÃO: ESTAMOS SOBRESCREVENDO A VARIÁVEI df
df = df.drop_duplicates()
print(df.shape)

In [ ]:
# Estudando as colunas categóricas
print('Elementos em \'key\': ', df['key'].unique())
print('Elementos em \'audio_mode\': ', df['audio_mode'].unique())
print('Elementos em \'time_signature\': ', df['time_signature'].unique())

In [ ]:
# Remapeando as colunas categóricas
df['key'].replace({0: 'C', 1: 'C#/Db', 2: 'D', 3: 'D#/Eb', 4: 'E', 5: 'F', 6: 'F#/Gb', 7: 'G', 8: 'G#/Ab', 9: 'A', 10: 'A#/Bb', 11: 'B'}, inplace = True)
df['audio_mode'].replace({0: 'minor', 1: 'major'}, inplace = True)
df['time_signature'].replace({3: '3/4', 4: '4/4', 5: '5/4', 0: '0?', 1: '1?'}, inplace = True)

In [ ]:
# Vendo se está tudo em ordem
df

In [ ]:
# Dando nome a algumas features
# feat_quant = ['song_duration_ms', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'audio_valence']
feat_quant = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'audio_valence']
feat_quali = ['key', 'audio_mode', 'time_signature']

In [ ]:
# Correlação LINEAR entre os atributos e a resposta?
plt.figure(figsize=(10, 10))
mask = np.triu(np.ones_like(df[[*feat_quant, 'song_popularity']].corr(), dtype=bool))
sns.heatmap(df[[*feat_quant, 'song_popularity']].corr(method = 'pearson'), annot=True, mask=mask, vmin=-1, vmax=1, cmap = "vlag")
plt.title('Correlações LINEARES entre os atributos e a resposta')
plt.show()

In [ ]:
# Correlação MAIS GERAL entre os atributos?
# https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
plt.figure(figsize=(10, 10))
mask = np.triu(np.ones_like(df[[*feat_quant, 'song_popularity']].corr(), dtype=bool))
sns.heatmap(df[[*feat_quant, 'song_popularity']].corr(method = 'spearman'), annot=True, mask=mask, vmin=-1, vmax=1, cmap = "vlag")
plt.title('Correlações MAIS GERAIS entre os atributos')
plt.show()

In [ ]:
# Enxergando pares de covariáveis
g = sns.pairplot(df[[*feat_quant, 'song_popularity']])
plt.title('Gráfico de dispersão em pares')
# g.map_upper(sns.kdeplot, levels=4, color=".2")
plt.show()

In [ ]:
# Relação entre atributos qualitativos e a resposta
plt.figure(figsize=(10, 5))
sns.scatterplot(data=df, x=df['key'], y='song_popularity', hue='song_popularity', legend = False)
plt.show()

In [ ]:
# Relação entre atributos qualitativos e a resposta
plt.figure(figsize=(3, 5))
sns.scatterplot(data=df, x=df['audio_mode'], y='song_popularity', hue='song_popularity', legend = False)
plt.show()

In [ ]:
# Relação entre atributos qualitativos e a resposta
plt.figure(figsize=(10, 5))
sns.scatterplot(data=df, x=df['time_signature'], y='song_popularity', hue='song_popularity', legend = False)
plt.show()

## Parte 1

a) Valide o desempenho de modelos de regressão linear e KNN, usando como atributos somente as variáveis quantitativas. Mais especificamente, use o conjunto de teste para treinar modelos de regressão linear com e sem penalização (encontrando hiperparâmetros ótimos por validação cruzada) e um modelo de KNN (também encontrando hiperparâmetros ótimos por validação cruzada), e meça o seu desempenho no conjunto de teste.

## Parte 2

b) Valide o desempenho de modelos de regressão baseados em árvores, usando como atributos somente as variáveis quantitativas. Mais especificamente, use o conjunto de teste para treinar modelos de regressão baseados em árvores (encontrando hiperparâmetros ótimos por validação cruzada) e meça o seu desempenho no conjunto de teste.

## Parte 3

c) Refaça os itens a) e b), utilizando uma `Pipeline` para acoplar um `StandardScaler` a cada um dos modelos de regressão empregados.

d) Decida por um "melhor" modelo para prever a popularidade de uma música com base nos seus atributos acústicos, e justifique a sua escolha.